In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ID_Labels = pd.read_csv('ID Labels.csv')
India_Data = pd.read_csv('India Data.csv')
Indicator_Change_Hex = pd.read_csv('Hex Data.csv')
PC_Data = pd.read_csv('PC Data.csv')
State_Data = pd.read_csv('State Data.csv')

In [ ]:
ID_Labels.head()

In [ ]:
PC_IDs = [x for x in ID_Labels['PC ID'].values if str(x) != 'nan'][1:]
PCs = [x for x in ID_Labels['Unnamed: 1'].values if str(x) != 'nan'][1:]
States = [x for x in ID_Labels['Unnamed: 3'].values if str(x) != 'nan'][1:]

IDsToPC = {}

for i in range(len(PC_IDs)):
    IDsToPC[int(PC_IDs[i])] = (PCs[i], States[i])

print(IDsToPC) #Creates a dictionary mapping PC IDs to a tuple containing both the PC and State names

In [ ]:
StateIDs = PC_IDs = [x for x in ID_Labels['State ID'].values if str(x) != 'nan'][1:]
IDsToState = {}
StateToID = {}

for i in range(len(StateIDs)):
    IDsToState[int(StateIDs[i])] = States[i]
    StateToID[States[i]] = int(StateIDs[i])

print(IDsToState) #Creates 2 dictionaries: one mapping State ID to State name and one mapping State name to State ID

In [ ]:
IndicatorIDs = [x for x in ID_Labels['Indicator ID'].values if str(x) != 'nan'][1:]
Indicators = [x for x in ID_Labels['Unnamed: 5'].values if str(x) != 'nan'][1:]
IDsToIndicator = {}
for i in range(len(IndicatorIDs)):
    IDsToIndicator[IndicatorIDs[i]] = Indicators[i]

print(IDsToIndicator) #Creates a dictionary mapping Indicator IDs to the name of the Indicator

In [ ]:
PCList = [IDsToPC[i][0] for i in IDsToPC]
StateList = [IDsToPC[i][1] for i in IDsToPC]
PCIDsList = [i for i in IDsToPC]

data = {'ID': PCIDsList, 'PC': PCList, 'State': StateList} #Creates a dictionary that will be used for final csv file
#print(PCList)
#print(StateList)

In [ ]:
IndiaIndicators = {}
IndicatorDirection = {}

for index, row in India_Data.iterrows():
    IndicatorDirection[row['Indicator ID']] = row['Indicator Direction']
    if str(row['All India Prevalence 2021']) != 'nan':
        if row['Indicator Direction'] == 1:
            IndiaIndicators[row['Indicator ID']] = row['All India Prevalence 2021']
        else:
            IndiaIndicators[row['Indicator ID']] = -1*row['All India Prevalence 2021']

#print(IndiaIndicators)
print(IndicatorDirection)

#Creates a dictionary IndicatorDirection that maps Indicator IDs to the direction of the dictionary
#Creates second dictionary IndiaIndicators that carries all indicators of India, adjusting for thier directions

In [ ]:
StateIndicators = {}
for i in IDsToState:
    IndividualStateData = State_Data[State_Data['State ID'] == int(i)]
    IndividualStateIndicators = {}
    for index, row in IndividualStateData.iterrows():
        if str(row['State Prevalence 2021']) != 'nan':
            if IndicatorDirection[row['Indicator ID']] == 1:
                IndividualStateIndicators[row['Indicator ID']] = row['State Prevalence 2021']
            else:
                IndividualStateIndicators[row['Indicator ID']] = -1*row['State Prevalence 2021']
    StateIndicators[IDsToState[i]] = IndividualStateIndicators

print([i for i in StateIndicators]) #Creates dictionary that carries all indicators for each state of India, adjusting for thier directions

In [ ]:
def common_and_greater(PC, larger):
    '''Returns how many indicators are common between the PC and State/India 
    dataset along with how many PC indicators are greater'''
    common = 0
    greater = 0
    for Id in IndicatorIDs:
        if int(Id) in larger and int(Id) in PC:
            common += 1
            if PC[int(Id)] > larger[int(Id)]:
                greater += 1
    return common, greater

In [ ]:
IndiaCommonAll = []
IndiaGreaterAll = []
StateCommonAll = []
StateGreaterAll = []
for i in [x for x in PC_Data['PC ID'].unique() if str(x) != 'nan']:
    IndividualPCData = PC_Data[PC_Data['PC ID'] == i]
    PCDict = {}
    #IndiaCommon = 0
    #IndiaGreater = 0
    #StateCommon = 0
    #StateGreater = 0
    for index, row in IndividualPCData.iterrows():
        if IndicatorDirection[row['Indicator ID']] == 1:
            PCDict[row['Indicator ID']] = row['Prevalence 2021']
        else:
            PCDict[row['Indicator ID']] = -1*row['Prevalence 2021']
    dataForState = StateIndicators[IDsToPC[i][1]] #State data for the PC
    '''
    for Id in IndicatorIDs:
        if int(Id) in IndiaIndicators and int(Id) in PCDict:
            IndiaCommon += 1
            if PCDict[int(Id)] > IndiaIndicators[int(Id)]:
                IndiaGreater += 1
    for Id in IndicatorIDs:
        if int(Id) in dataForState and int(Id) in PCDict:
            StateCommon += 1
            if PCDict[int(Id)] > dataForState[int(Id)]:
                StateGreater += 1
    '''
    IndiaCommon, IndiaGreater = common_and_greater(PCDict, IndiaIndicators)
    StateCommon, StateGreater = common_and_greater(PCDict, dataForState)
    IndiaCommonAll.append(IndiaCommon)
    IndiaGreaterAll.append(IndiaGreater)
    StateCommonAll.append(StateCommon)
    StateGreaterAll.append(StateGreater)

data['India Common'] = IndiaCommonAll
data['India Greater'] = IndiaGreaterAll
data['State Common'] = StateCommonAll
data['State Greater'] = StateGreaterAll #Finds how many indicators are in common and greater between the PC and State/India
'''
    if i==101 or i==102:
        print(IndiaCommon)
        print(IndiaGreater)
        print(StateCommon)
        print(StateGreater)
'''

In [ ]:
print(data['India Greater'])

In [ ]:
#PC_Data.head() 
#IndicatorDirection[row['Indicator ID']]
'''
highestList = []
lowestList = []
for i in IDsToPC:
    lowest = []
    highest = []
    ranks = {}
    dataForPC = PC_Data[PC_Data['PC ID'] == i]
    for index, row in dataForPC.iterrows():
        if str(row['Prevalence Change']) != 'nan':
            if IndicatorDirection[row['Indicator ID']] == 1:
                ranks[row['Indicator ID']] = (row['Prevalence Change'])
            else:
                ranks[row['Indicator ID']] = (row['Prevalence Change']*-1)
    sortedRanks = sorted(ranks.items(), key=lambda x:x[1])
    for i in range(len(sortedRanks)):
        print(sortedRanks)
        if i <= 9:
            lowest.append(IDsToIndicator[str(int(sortedRanks[i][0]))])
        elif i >= len(sortedRanks)-10:
            highest.append(IDsToIndicator[str(int(sortedRanks[i][0]))])
    highestList.append(highest)
    lowestList.append(lowest)

data['Improvement Indicators'] = highestList
data['Worsening Indicators'] = lowestList
'''

In [ ]:

highestDict = {}
lowestDict = {}
for i in IDsToPC:
    ranks = {}
    dataForPC = PC_Data[PC_Data['PC ID'] == i]
    for index, row in dataForPC.iterrows():
        if str(row['Prevalence Change']) != 'nan':
            if IndicatorDirection[row['Indicator ID']] == 1:
                ranks[row['Indicator ID']] = (row['Prevalence Change'])
            else:
                ranks[row['Indicator ID']] = (row['Prevalence Change']*-1)
    sortedRanks = sorted(ranks.items(), key=lambda x:x[1])
    for j in range(len(sortedRanks)):
        if j <= 9:
            if j+1 in lowestDict:
                lowestDict[j+1].append(IDsToIndicator[str(int(sortedRanks[j][0]))])
            else:
                lowestDict[j+1] = [IDsToIndicator[str(int(sortedRanks[j][0]))]]
        elif j >= len(sortedRanks)-10:
            if (len(sortedRanks) - j) in highestDict:
                highestDict[len(sortedRanks) - j].append(IDsToIndicator[str(int(sortedRanks[j][0]))])
            else:
                highestDict[len(sortedRanks) - j] = [IDsToIndicator[str(int(sortedRanks[j][0]))]]

for i in highestDict:
    data['Improvement Indicators ' + str(i)] = highestDict[i]

for i in lowestDict:
    data['Worsening Indicators ' + str(i)] = lowestDict[i]
#Finds the indicators showing the most or least improvement

In [ ]:
Highest_Improvement = []
Improvement = []
Worsened = []
Extremely_Worsened = []
TotalInds = []

for i in IDsToPC:
    hi = 0
    imp = 0
    w = 0
    ew = 0
    dataForPC = PC_Data[PC_Data['PC ID'] == i]
    for index, row in dataForPC.iterrows():
        if row['Prevalence Change Category '] == 1:
            hi += 1
        elif row['Prevalence Change Category '] == 2:
            imp += 1
        elif row['Prevalence Change Category '] == 3:
            w += 1
        elif row['Prevalence Change Category '] == 4:
            ew += 1
    Highest_Improvement.append(hi)
    Improvement.append(imp)
    Worsened.append(w)
    Extremely_Worsened.append(ew)
    TotalInds.append(hi+imp+w+ew)

data['Highest Improvement'] = Highest_Improvement
data['Improvement'] = Improvement
data['Worsened'] = Worsened
data['Extremely Worsened'] = Extremely_Worsened
data['Indicators with Change Data'] = TotalInds

#Finds the number of indicators of each category along with the total

In [ ]:
for i in data:
    if len(data[i]) != 543:
        print(len(data[i]), i)

In [ ]:
tempData = pd.DataFrame(data)

In [ ]:
tempData.head()
IDToImprovement = {}

for index, row in tempData.iterrows():
    IDToImprovement[row['ID']] = (row['Highest Improvement'] + row['Improvement']) / row['Indicators with Change Data']

sortedImprovement = sorted(IDToImprovement.items(), key=lambda x:x[1])
sortedImprovement.reverse()
improvementNums = [i[1] for i in sortedImprovement]
#print(improvementNums)

list1 = []
SharedBy = []

for i in set(improvementNums):
    list1.append(improvementNums.count(i))

for i in list1:
    for j in range(i):
        SharedBy.append(i)

#print(list1)

tempList = [0]

for i in list1:
    tempList.append(i)

improvementRanks = []

for i in range(1,len(tempList)):
    for j in range(tempList[i]):
        improvementRanks.append(sum(tempList[0:i])+1)

#print(SharedBy)

improvementDict = {}

for i in range(len(sortedImprovement)):
    improvementDict[[i[0] for i in sortedImprovement][i]] = (improvementRanks[i], SharedBy[i])

#print(improvementDict)

myKeys = list(improvementDict.keys())
myKeys.sort()
sorted_dict = {i: improvementDict[i] for i in myKeys}
 
print(sorted_dict)

data['Ranks for Improvement'] = [sorted_dict[i][0] for i in sorted_dict]
data['Ranks for Improvement Shared by'] = [sorted_dict[i][1] for i in sorted_dict]

#Finds the rank for each indicator along with how many PCs share that rank

In [ ]:
India_Data.head()

for index, row in India_Data.iterrows():
    IndicatorIndiaData = []
    for j in range(len(data['ID'])):
        IndicatorIndiaData.append(row['All India Prevalence 2021'])
    data[IDsToIndicator[str(int(row['Indicator ID']))] + ' India'] = IndicatorIndiaData

#Puts the data for India into every row of the new dataset

In [ ]:
print(IndicatorIDs)
for id in IndicatorIDs:
    PCHex = []
    PCPrevalence = []
    PCChange = []
    StateInds = []
    for pc in IDsToPC:
        sub = PC_Data[PC_Data['PC ID'] == pc]
        rowForPC = sub[sub['Indicator ID'] == int(id)]
        for index, row in rowForPC.iterrows():
            PCPrevalence.append(row['Prevalence 2021'])
            PCChange.append(row['Prevalence Change'])
            if row['Prevalence Change Category '] == 1:
                PCHex.append('#2E7265')
            elif row['Prevalence Change Category '] == 2:
                PCHex.append('#6D9F95')
            elif row['Prevalence Change Category '] == 3:
                PCHex.append('#E87D7F')
            elif row['Prevalence Change Category '] == 4:
                PCHex.append('#DF4649')
            else:
                PCHex.append('nan') 
            state = StateToID[IDsToPC[pc][1]]
            sub = State_Data[State_Data['State ID'] == state]
            rowForState = sub[sub['Indicator ID'] == int(id)]
            for i, r in rowForState.iterrows():
                StateInds.append(r['State Prevalence 2021'])
        data[IDsToIndicator[id] + ' State'] = StateInds
        data[IDsToIndicator[id] + ' PC'] = PCPrevalence
        data[IDsToIndicator[id] + ' PC Change'] = PCChange
        data[IDsToIndicator[id] + ' PC Hex'] = PCHex
#Puts the state indicator values and PC indicator values into the dataset

In [ ]:
for i in data:
    if len(data[i]) != 543:
        print(i, len(data[i]))

In [ ]:
df = pd.DataFrame(data)
df = df.drop('ID', axis=1)

In [ ]:
df.head(543)

In [ ]:
df.to_csv('new_data.csv', index=False)